# Homework 2
Zach Bogart, Josh Feldman

- In this homework, we are trying to create movie recommendations for users based on the MovieLens dataset. We want to create recommendations that are relevant to the users, giving new movies thay have not seen, but might enjoy since other users with similar watch patterns have rated them highly.
- We explored two models of collaborative filtering: one model-based and one neighborhood-based. We used non-negative matrix factorization (model-based) and k-nearest neighbors (neighborhood-based). We are interested in seeing how the models differ in accuracy metrics and what sacrifices may have to be made to achieve those results.
- For our accuracy metrics, we used root mean squared error and mean absolute error. We used both of these metrics because they weighted outliers differently, giving a different sense of how accurate our results are. RMSE weights outliers more heavily. We chose RMSE as our primary accuracy metric.



## Our Process (the python script)

- To test the ability of the model to give good predictions to users, we needed to get the best parameters for the model. To do this, we did cross validation using gridSearchCV to test the effect different hyperparameters had on the model. We split the cross validation into separate grid searches since the models take a long time to run (due to the size of the dataset). 
- After grid search, we save off the best parameters.
- With the best parameters, we systematically tested how the size of the training data affected results, collecting accuracy metrics and runtime figures.
- Finally, we used the best hyperparameters to train the models and evaluate them on the test data. We then saved off these models to be used to make recommendations.

- With this, we have a setup where we can pass in a model, cross-validate the desired hyperparameters, and report accuracy metrics based on those choices to find the best setup. Below we will show the results.

- **NOTE**: This notebook expects the results of the script. We saved off the results to be used. However, you can run the script if you want (it takes a wahile on the large dataset).

## Read in Script Results

In [ ]:
import os
import pickle
import matplotlib.pyplot as plt

In [ ]:
resultFiles = {}

for filename in os.listdir('results'):
    if ".sav" not in filename:
        infile = open("results/{}".format(filename), 'rb')
        result = pickle.load(infile)
        resultFiles[filename] = result
        infile.close()

Our resultsFiles contains 3 separate results:
- Files starting with 'gridSearchResults' are results from parameter tuning. We saved not only the best parameters and metrics, but also the entire cv_results from the gridSearch
- Files starting with 'dataSizeResults' are lists of results from running our models with different data sizes
- Files starting with 'finalResults' are results from the final run of our models. 

## Matrix Factorization Method

In the NMF setup, there are a set of parameters we can tune:

- n_epochs: number of iterations of stochastic gradient descent
- n_factors: number of intermediate factors
- reg_pu, reg_qi: regularization coefficients
- biased: whether to use baselines

Let's see how they affect the accuracy metrics. We have created a function to create the plots of each hyperparamater for the model

In [ ]:
def plotParameterResults(gridSearchFiles, modelType):
    bestParams = {}
    for gridSearchKey in gridSearchFiles.keys():
        if "gridSearchResults{}".format(modelType) in gridSearchKey:
            periodIndex = gridSearchKey.find('.')
            underscoreIndex = gridSearchKey.find('_')
            param = gridSearchKey[underscoreIndex + 1:periodIndex]
        
            if param == 'random_state' or param == 'verbose':
                continue

            gridSearch = gridSearchFiles[gridSearchKey]
            
            bestParamValue = gridSearch['bestParams'][param]
            bestParams[param] = bestParamValue

            for index, resultType in enumerate(['mean_test_mae', 'mean_test_rmse']):
                x = [str(result) for result in gridSearch["cv_results"]['param_{}'.format(param)]]
                y = [result for result in gridSearch["cv_results"][resultType]]

                plt.bar(x, y, color='black')
                plt.title('Matrix Factorization Model, {}'.format(param))
                plt.xlabel(param)
                plt.ylabel(resultType)
                plt.show()
                plt.close()
    return bestParams

In [ ]:
bestParamsNMF = plotParameterResults(resultFiles, 'NMF')

In [ ]:
print('The best params using rmse are:')
for key, value in bestParamsNMF.items():
    print('{}: {}'.format(key, value))

## Neighborhood Method

In [ ]:
bestParamsKNN = plotParameterResults(resultFiles, 'KNN')

In [ ]:
print('The best params using rmse are:')
for key, value in bestParamsKNN.items():
    print('{}: {}'.format(key, value))

## Tune Sample Size

In [ ]:
# def plotSampleSizes(gridSearchFiles):
filename = "dataSizeResultsKNN.p"
resultList = resultFiles[filename]
timeKNN = [item['time'] for item in resultList]
rmseKNN = [item['rmse'] for item in resultList]
numRatingsKNN = [item['numRatings'] for item in resultList]

filename = "dataSizeResultsNMF.p"
resultList = resultFiles[filename]
timeNMF = [item['time'] for item in resultList]
rmseNMF = [item['rmse'] for item in resultList]
numRatingsNMF = [item['numRatings'] for item in resultList]
        

### NMF

In [ ]:
plt.plot(numRatingsNMF, rmseNMF, 'o', color='black')
plt.title('Matrix Factorization Model')
plt.xlabel('Data Sample Percentage')
plt.ylabel('rmse')
plt.show()
plt.close()

plt.plot(numRatingsNMF, timeNMF, 'o', color='black')
plt.title('Matrix Factorization Model')
plt.xlabel('Data Sample Percentage')
plt.ylabel('Runtime to Train')
plt.show()
plt.close()

### KNN

In [ ]:
plt.plot(numRatingsKNN, rmseKNN, 'o', color='black')
plt.title('Matrix Factorization Model')
plt.xlabel('Data Sample Percentage')
plt.ylabel('rmse')
plt.show()
plt.close()

plt.plot(numRatingsKNN, timeKNN, 'o', color='black')
plt.title('Matrix Factorization Model')
plt.xlabel('Data Sample Percentage')
plt.ylabel('Runtime to Train')
plt.show()
plt.close()

**RESULTS**:

- For NMF, as you add more data, the RMSE exponentially decreases. Meanwhile, the runtime increases linearly as you add more data. This makes sense since more data will of course take longer to train. However, we can see that one of our choices in implimenting this model. At a certain point, adding data no longer changes the accuracy metric by much (for example, doubling the data from 10K to 20K makes a noticable difference, while doubling from 30K to 60K makes much less of a difference). This should be taken into account when deciding how much data to use to train the model.

- If the goal is to make the model quick to train, you could sacrifice some accuracy for a large gain in runtime.

- A similar result is seen in the knn model. Again, it should be considered when deploying whether to sacrifice a small amount of accuracy for a gain in runtime.

## Final Model

### NMF vs. KNN

In [ ]:
resultsNMF = resultFiles['finalResultsNMF.p']
resultsKNN = resultFiles['finalResultsKNN.p']
models = ['NMF', 'KNN']
rmseScores = resultsNMF['rsme'], resultsKNN['rsme']
maeScores = resultsNMF['mae'], resultsKNN['mae']

plt.bar(models, rmseScores, color='black')
plt.title('Matrix Factorization vs KNN')
plt.xlabel('Models')
plt.ylabel('rmse')
plt.show()
plt.close()

plt.bar(models, maeScores, color='black')
plt.title('Matrix Factorization vs KNN')
plt.xlabel('Models')
plt.ylabel('mae')
plt.show()
plt.close()

## Conclusion

Overall, from our final results we can see that our KNN model seems to outperform our Matrix Factorization model, both in our accuracy metrics and in the time to train. So, for deploying this model, we recommend (pun intended) using the knn model since it is much faster to train and gives better accuracy for recommendations related to the users. 